In [ ]:
%pip install selenium
%pip install pandas
%pip install pandas

In [1]:
import sys
import os
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import tqdm
from tqdm.notebook import tqdm


In [2]:
#데이터 불러오기 
name = pd.read_csv('movie_list.csv')

In [ ]:
#실수 형인지 확인 
def is_valid_float(element: str) -> bool:
    try:
        float(element)
        return True
    except ValueError:
        return False

In [45]:
inp =name["title"]
#추출할 영화 개수 선택 
inp2 = inp[0:10]
inp2

0              두 여인
1                이창
2                계모
3                롤라
4    2001 스페이스 오디세이
5               수색자
6         티파니에서 아침을
7             악마의 씨
8            거미집의 성
9                휴일
Name: title, dtype: object

In [103]:
#크롤링 결과를 데이터 프레임으로 저장 
result = pd.DataFrame(data=None, columns=("title",'rating','country','runningtime','smry',"star", 'm_ratio',"review"))

driver = webdriver.Chrome()#, options=options)
driver.get('http://www.naver.com')
time.sleep(2)

for title in tqdm(inp2):
    try:
        element = driver.find_element(By.ID,'query')  
        element.send_keys("영화 ", title)
        element.submit()
        driver.implicitly_wait(2)
        
        element = driver.find_element(By.CLASS_NAME,'_au_movie_content_wrap')
        
        #변수 초기화 
        star = ''# 평점
        rating = ''# 관람등급
        country = ''# 국가 
        runningtime = ''# 러닝타임
        m_ratio = '' #관람객 성비 
        smry = ''
        b =[]

        # 평점
        try:
            informations = driver.find_elements(By.CSS_SELECTOR, 'div.cm_content_area._cm_content_area_info > div > div.detail_info > dl > div')
            for i in range(1,len(informations)+1):
                info = driver.find_element(By.CSS_SELECTOR, 'div.cm_content_area._cm_content_area_info > div > div.detail_info > dl > div:nth-child(%d) > dt'%i).text
                if info == '평점':
                    star = driver.find_element(By.CSS_SELECTOR, 'div.cm_content_area._cm_content_area_info > div > div.detail_info > dl > div:nth-child(%d) > dd'%i).text
        except NoSuchElementException:
            pass
                
        # 메타데이터  
        driver.find_element(By.CSS_SELECTOR, 'div.cm_content_wrap > div.cm_content_area._cm_content_area_info > div > div.detail_info > div > a').click()
        driver.implicitly_wait(2)
        try: 
            informations = driver.find_elements(By.CSS_SELECTOR, 'div.cm_content_wrap > div.cm_content_area._cm_content_area_info > div > div.detail_info > dl > div')
            for i in range(1,len(informations)+1):
                info = driver.find_element(By.CSS_SELECTOR, 'div.cm_content_area._cm_content_area_info > div > div.detail_info > dl > div:nth-child(%d) > dt'%i).text
                if info == '등급':
                    rating = driver.find_element(By.CSS_SELECTOR, 'div.cm_content_area._cm_content_area_info > div > div.detail_info > dl > div:nth-child(%d) > dd'%i).text
                elif info == '국가':
                    country = driver.find_element(By.CSS_SELECTOR, 'div.cm_content_area._cm_content_area_info > div > div.detail_info > dl > div:nth-child(%d) > dd'%i).text
                elif info == '러닝타임':
                    runningtime = driver.find_element(By.CSS_SELECTOR, 'div.cm_content_area._cm_content_area_info > div > div.detail_info > dl > div:nth-child(%d) > dd'%i).text 
        except:
           pass
        
        # 줄거리 
        try:
           smry = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[3]/div[2]/div[2]/div/div[2]/p').text
        except NoSuchElementException:
           pass

        driver.back()
        # 관람평으로 이동 
        try:            
            driver.find_element(By.CSS_SELECTOR, 'div.cm_content_wrap > div.cm_content_area._cm_content_area_graph_donut > div > div.middle_title > a').click()
            driver.implicitly_wait(2)
            m_ratio = driver.find_element(By.CSS_SELECTOR,'div > div > ul > li.type_male > span > em').text
            time.sleep(0.5)
            review_list = driver.find_element(By.CSS_SELECTOR, 'div.lego_review_list._scroller')
            driver.execute_script("arguments[0].scrollBy(0, 1000)",review_list)

            # 관람평 크롤링 
            x = driver.find_elements(By.CSS_SELECTOR, 'div.lego_review_list._scroller > ul > li')
            for i in range(1,len(x)):
                a = driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller > ul > li:nth-child(%d) > div.area_review_content > div > span.desc._text'%(i)).text
                a = a.replace('\n','')
                a = a.replace('#','')
                b.append(a)
        except NoSuchElementException:
            pass
        
        #데이터 저장 
        new = pd.DataFrame({"title":[title],'rating':[rating],'country':[country],'runningtime':[runningtime],'smry':[smry],"star":[star], 'm_ratio':[m_ratio],"review":[b]})
        result = pd.concat([result, new], ignore_index=True)

        driver.get('http://www.naver.com')
    except NoSuchElementException:
        new = pd.DataFrame({"title":[title],'rating':[rating],'country':[country],'runningtime':[runningtime],'smry':[smry],"star":[star], 'm_ratio':[m_ratio],"review":[b]})
        result = pd.concat([result, new], ignore_index=True)
        driver.get('http://www.naver.com')
        
driver.close()

  0%|          | 0/10 [00:00<?, ?it/s]

In [104]:
# 결과 확인 
result

,title,rating,country,runningtime,smry,star,m_ratio,review
0,두 여인,,미국,123분,명문대를 나와 변호사가 된 부잣집딸 힐러리(Hillary Whitney Essex:...,,,[]
1,이창,15세 이상 관람가,미국,112분,사진작가 제프는 카레이싱 촬영 도중 다리를 다쳐 휠체어에 의지한 채 자신의 방에서 ...,8.95,,"[이것이 전설의 이창이로구나! 단 하나의 장소에서 이 모든 것을 이루어 내다니, 놀..."
2,계모,,대한민국,110분,초혼에 실패한 그녀는 모 재벌의 후처로 들어가서 열성을 다해 가정을 이끌어 간다. ...,,,[]
3,롤라,12세 이상 관람가,프랑스,115분,직장까지 포기한 채 춤에 대한 열정으로 가득찬 롤라는 매번 오디션에서 떨어지며 현실...,8.37,46,"[앞부분은 지루지루.. 갈수록 볼만한 ㅋ 롤라도 이스마한도 이쁜걸 ㅋ, 댄스란 무엇..."
4,2001 스페이스 오디세이,12세 이상 관람가,"영국, 미국",150분,인류에게 문명의 지혜를 가르쳐 준 검은 돌기둥의 정체를 밝히기 위해서 목성으로 향하...,8.91,,[]
5,수색자,12세 이상 관람가,미국,119분,남북 전쟁 당시 남부 동맹에 군인으로 참전해 3년간 집을 떠나 있던 전직 보안관 이...,8.36,83,"[영화 같은 것은 진화하는 게 아니다. 2020년이 와도, 2100년이 와도, 30..."
6,티파니에서 아침을,15세 이상 관람가,미국,115분,"소매가 치렁한 이브닝 드레스, 얼굴을 반이나 가린 검은 안경. 그녀는 티파니 보석상...",8.38,,"[보석같은 영화입니다., 진짜 최고의 영화, 살면서 여러 번 보게 될 영화, 클래식..."
7,악마의 씨,,,,,,,[]
8,거미집의 성,,,,,,,[]
9,휴일,,일본,14분,"어느 시골의 휴양지, 주변에는 마치 장지문에 그려진 듯한 커다란 소나무가 있고, 케...",,,[]


In [85]:
# 결과 저장 
result.to_csv('review_naver.csv', index =False, encoding="UTF-8-sig")